In [1]:
!pip install gdown
# !gdown "https://drive.google.com/uc?id=1-Ct4OcUrx4TS8mdna3xFcqltTAfok1GO"
# !gdown "https://drive.google.com/uc?id=1Ucc4TDBiw5yl6HKdTNyEcN-Prv044DJU"

In [2]:
!gdown "https://drive.google.com/uc?id=15L-jmGrls36U3odT_qElAmc5-6huEJuh"
!gdown "https://drive.google.com/uc?id=1cwXz_EMm6pZTEzTBLvJCYDU1j8HUY2Bd"


Downloading...
From: https://drive.google.com/uc?id=15L-jmGrls36U3odT_qElAmc5-6huEJuh
To: /content/protein_feat_one.pkl
100% 15.6M/15.6M [00:00<00:00, 87.8MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1cwXz_EMm6pZTEzTBLvJCYDU1j8HUY2Bd
From (redirected): https://drive.google.com/uc?id=1cwXz_EMm6pZTEzTBLvJCYDU1j8HUY2Bd&confirm=t&uuid=406ed284-f643-4f12-9eff-7d5e732c8cc2
To: /content/protein_feat_r.pkl
100% 400M/400M [00:09<00:00, 41.6MB/s]


In [3]:
!git clone https://github.com/sharzil1994/TranP-B-site.git

fatal: destination path 'TranP-B-site' already exists and is not an empty directory.


In [4]:
import os
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Conv1D, Conv2D,BatchNormalization, MaxPooling1D,MaxPooling2D,LayerNormalization, Dropout, Flatten, Dense, concatenate ,AveragePooling1D
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.metrics import binary_accuracy
from sklearn.metrics import confusion_matrix, recall_score, roc_curve, roc_auc_score, auc,average_precision_score,matthews_corrcoef
import math
import tensorflow as tf
from tensorflow.keras import losses
from sklearn.utils import compute_class_weight
import random
import itertools
import pickle
np.random.seed(seed=21)
import math

In [5]:
win_size=5
emb=''

In [6]:
def windowing(features,w_size):
    to=0
    for index in features:
        b=np.shape(index)[0]
        to=to+b



    a=features[0]
    fea_len=np.shape(a)[1]
    finalout1=np.zeros([to,w_size,fea_len],'float')

    l=0
    for i in range(0,len(features)):
        temp_features=features[i]

        for j in range( 0, np.shape(temp_features)[0]):


            for k in range(0,w_size):

                k1=int(j+k-((w_size-1)/2))

                if k1<0 or k1 > np.shape(temp_features)[0]-1:
                    pass
                else:
                    finalout1[l,k,:]=temp_features[k1,:]
            l=l+1
    finalout1=finalout1.reshape((finalout1.shape[0], finalout1.shape[1],finalout1.shape[2], 1))
    return finalout1

def label( labels):
  final_label=labels

  to=0
  for i in range(0,len(final_label)):
      temp_label=final_label[i]
      for j in range(0,len(temp_label)):
          to=to+1


  finallabel=np.zeros([to],'int')

  l=0
  for i in range(0,len(final_label)):
      # print(i)

      temp_label=final_label[i]

      for j in range( 0, len(temp_label)):


          finallabel[l]=temp_label[j]
          l=l+1

  return  finallabel


In [7]:
pathdata='./TranP-B-site/datasets/'

with open(pathdata+'Train_335.pkl', 'rb') as file:
    train = pickle.load(file)
with open(pathdata+'Test_60.pkl', 'rb') as file:
    test = pickle.load(file)
with open(pathdata+'Test_20_new.pkl', 'rb') as file:
    test_20 = pickle.load(file)
with open(pathdata+'UBtest_31.pkl', 'rb') as file:
    test_UB_31 = pickle.load(file)
with open(pathdata+'Btest_31.pkl', 'rb') as file:
    test_B_31 = pickle.load(file)
with open(pathdata+'Test_70.pkl', 'rb') as file:
    test_70 = pickle.load(file)

all_data=dict()
for index in train.keys():
    all_data[index]=train[index]
for index in test.keys():
    all_data[index]=test[index]
for index in test_20.keys():
    all_data[index]=test_20[index]
for index in test_UB_31.keys():
    all_data[index]=test_UB_31[index]
for index in test_B_31.keys():
    all_data[index]=test_B_31[index]
for index in test_70.keys():
    all_data[index]=test_70[index]


seq_all=dict()
for index in all_data.keys():
    temp=all_data[index]
    seq_all[index]=temp[0]
label_all=dict()
for index in all_data.keys():
    temp=all_data[index]
    label_all[index]=temp[1]

with open(os.path.join(f'protein_feat_r.pkl'), 'rb') as file:
    protein_feat_r = pickle.load(file)
    print('loaded protein_feat_r')

with open(os.path.join('protein_feat_one.pkl'), 'rb') as file:
    protein_feat_one = pickle.load(file)
    print('loaded protein_feat_one')

new_dataset=dict()
for index in train.keys():
    new_dataset[index]=train[index]
for index in train.keys():
    for index1 in test_70.keys():
        if index == index1:
            del new_dataset[index1]
train=new_dataset








loaded protein_feat_r
loaded protein_feat_one


In [8]:
def data_processing(data_in_2,data_in_3,data_in_4,data_in_5):

  data_in=dict()
  data_in_one=dict()
  data_in_label=dict()

  for index in data_in_2.keys():
      data_in[index]=data_in_3[index]
      data_in_one[index]=data_in_4[index]
      data_in_label[index]=data_in_5[index]

  data_in=np.array(list(data_in.items()),dtype=object)[:,1]
  data_in_one=np.array(list(data_in_one.items()),dtype=object)[:,1]
  data_in_label=np.array(list(data_in_label.items()),dtype=object)[:,1]

  data_in=windowing(data_in,win_size)# transformer features
  data_in_one=windowing(data_in_one,win_size)#one hot  encoding features
  data_in_label=label(data_in_label) # lables
  return(data_in,data_in_one,data_in_label )




In [9]:
prot_test_60,prot_test_60_one,label_test_60=data_processing(test,protein_feat_r,protein_feat_one,label_all)
prot_train_335,prot_train_335_one,label_train_335=data_processing(train,protein_feat_r,protein_feat_one,label_all)
prot_test_20,prot_test_20_one,label_test_20=data_processing(test_20,protein_feat_r,protein_feat_one,label_all)
prot_test_UB_31,prot_test_UB_31_one,label_test_UB_31=data_processing(test_UB_31,protein_feat_r,protein_feat_one,label_all)
prot_test_B_31,prot_test_B_31_one,label_test_B_31=data_processing(test_B_31,protein_feat_r,protein_feat_one,label_all)
prot_test_70,prot_test_70_one,label_test_70=data_processing(test_70,protein_feat_r,protein_feat_one,label_all)




In [10]:
class model_class():
    def __init__(self,w_size):

        self.w_size=w_size


    def model_cnn_8(self):

        inputs = Input(shape = (self.w_size,1024,1))
        inputs2 = Input(shape = (self.w_size,20,1))


        c1 = Conv2D(32,(7,7), strides=(1,1), activation='relu', input_shape=(self.w_size, 1024,1),padding="same", kernel_regularizer=regularizers.l2(0.1))(inputs)


        c1 = LayerNormalization()(c1)
        c1 = Dropout(0.3,seed = 21)(c1)

        c1 = Conv2D(16,(5,5), strides=(1,1), activation='relu',padding="same", kernel_regularizer=regularizers.l2(0.05))(c1)

        c1 = LayerNormalization()(c1)

        c1 = Dropout(0.5,seed = 21)(c1)

        c1=Flatten()(c1)

        fc1 = Dense(1024, activation='relu')(c1)

        fc1 = Dense(512, activation='relu')(fc1)

        fc1 = Dense(256, activation='relu')(fc1)

        fc1 = Dense(128, activation='relu')(fc1)


        p1 = Flatten()(inputs2)

        fc2 = Dense(64, activation='relu')(p1)
        fc2 = Dense(32, activation='relu')(fc2)
        fc2 = Dense(16, activation='relu')(fc2)


        fc1 =concatenate([fc1,fc2])


        fc1 = Dense(64, activation='relu')(fc1)

        fc1 = Dense(32, activation='relu')(fc1)

        fc1 = Dense(16, activation='relu')(fc1)

        fc1 = Dense(8, activation='relu')(fc1)

        fc2 = Dense(1, activation='sigmoid')(fc1)

        model1 = Model(inputs =[inputs,inputs2], outputs = [fc2])
        opt=SGD(learning_rate=0.00001, momentum = 0.99)

        model1.compile(loss='binary_crossentropy', optimizer= opt, metrics=['accuracy'])

        return model1

def calculateScore(X, y, model):

        i=0
        j=0
        for index in range(len(y)):
            if y[index]==0:
                i=i+1
            else :
                j=j+1

        score = model.evaluate(X,y)
        pred_y = model.predict(X)
        accuracy = score[1];
        tempLabel = np.zeros(shape = y.shape, dtype=np.int32)
        for i in range(len(y)):
            if pred_y[i] < 0.5:
                tempLabel[i] = 0;
            else:
                tempLabel[i] = 1;
        confusion = confusion_matrix(y, tempLabel)
        TN, FP, FN, TP = confusion.ravel()

        sensitivity = recall_score(y, tempLabel)
        specificity = TN / float(TN+FP)

        # MCC = ((TP*TN)-(FP*FN))/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
        MCC=matthews_corrcoef(y, tempLabel)
        F1Score = (2 * TP) / float(2 * TP + FP + FN)
        precision = TP / float(TP + FP)
        pred_y = pred_y.reshape((-1, ))
        ROCArea = roc_auc_score(y, pred_y)
        recall = TP / float(TP + FN)
        fpr, tpr, thresholds = roc_curve(y, pred_y)
        lossValue = None;

        y_true = tf.convert_to_tensor(y, np.float32)
        y_pred = tf.convert_to_tensor(pred_y, np.float32)
        PRAuc=average_precision_score(y, pred_y)
        return {'sn' : sensitivity, 'sp' : specificity, 'acc' : accuracy, 'MCC' : MCC, 'AUC' : ROCArea,'PRAuc':PRAuc, 'precision' : precision, 'F1' : F1Score,  'lossValue' : lossValue}


In [ ]:
model1=model_class(win_size)
model = model1.model_cnn_8()

print(model.summary())

class_weights = compute_class_weight(class_weight = "balanced",classes = np.unique(label_train_335),y = label_train_335)
class_weights = dict(enumerate(class_weights))

history = model.fit([prot_train_335,prot_train_335_one], label_train_335, batch_size = 64,
                class_weight=class_weights,epochs=60)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 5, 1024, 1)     │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 5, 1024, 32)    │          1,600 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization       │ (None, 5, 1024, 32)    │             64 │ conv2d[0][0]           │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 5, 1024, 32)    │              0 │ layer_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 5, 1024, 16)    │         12,816 │ dropout[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_1     │ (None, 5, 1024, 16)    │             32 │ conv2d_1[0][0]         │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 5, 1024, 16)    │              0 │ layer_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten (Flatten)         │ (None, 81920)          │              0 │ dropout_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_1             │ (None, 5, 20, 1)       │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 1024)           │     83,887,104 │ flatten[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_1 (Flatten)       │ (None, 100)            │              0 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 512)            │        524,800 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_4 (Dense)           │ (None, 64)             │          6,464 │ flatten_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 256)            │        131,328 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_5 (Dense)           │ (None, 32)             │          2,080 │ dense_4[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 128)            │         32,896 │ dense_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_6 (Dense)           │ (None, 16)             │            528 │ dense_5[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 144)            │              0 │ dense_3[0][0],         │
│                      

 Total params: 84,611,745 (322.77 MB)

 Trainable params: 84,611,745 (322.77 MB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/60
 19/991 ━━━━━━━━━━━━━━━━━━━━ 57:30 4s/step - accuracy: 0.8380 - loss: 1.9977

In [ ]:
print('test_60')
print(calculateScore([prot_test_60,prot_test_60_one], label_test_60, model))

In [ ]:
print('test_70')
print(calculateScore([prot_test_70,prot_test_70_one], label_test_70, model))

In [ ]:
print('test_UB_31')
print(calculateScore([prot_test_UB_31,prot_test_UB_31_one], label_test_UB_31, model))

In [ ]:
print('test_B_31')
print(calculateScore([prot_test_B_31,prot_test_B_31_one], label_test_B_31, model))

In [ ]:
print('test_20')
print(calculateScore([prot_test_20,prot_test_20_one], label_test_20, model))